# Synthetix V3


In [11]:
import os
import time
from synthetix import Synthetix
from synthetix.utils import wei_to_ether, ether_to_wei, format_wei, format_ether
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
snx = Synthetix(
    provider_rpc=os.getenv("NETWORK_421614_RPC"),
    private_key=os.getenv("PRIVATE_KEY"),
    cannon_config={
        "package": "synthetix-omnibus",
        "version": "latest",
        "preset": "octopus",
    },
)

2024-07-26 10:33:57,006 - INFO - Using private key signer: 0x3239a95A9262034ca28b9a03133775f716f119f8
2024-07-26 10:33:57,354 - INFO - Setting network_id from RPC chain_id: 421614
2024-07-26 10:33:58,501 - INFO - Loading cannon contracts for synthetix-omnibus:latest@octopus


KeyboardInterrupt: 

### Setup


In [ ]:
# get some contracts
weth = snx.contracts["WETH"]["contract"]

# USDC token
usdc_package = snx.contracts["usdc_mock_collateral"]["MintableToken"]
usdc = snx.web3.eth.contract(address=usdc_package["address"], abi=usdc_package["abi"])

# BTC token
btc_package = snx.contracts["btc_mock_collateral"]["MintableToken"]
btc = snx.web3.eth.contract(address=btc_package["address"], abi=btc_package["abi"])

In [ ]:
def status():
    """A function printing some useful information about the connected account"""
    eth_balance = snx.get_eth_balance()
    susd_balance = snx.spot.get_balance(market_name="sUSD")

    usdc_balance = usdc.functions.balanceOf(snx.address).call()
    usdc_balance = format_wei(usdc_balance, 6)

    btc_balance = btc.functions.balanceOf(snx.address).call()
    btc_balance = format_wei(btc_balance, 18)

    sbtc_balance = snx.spot.get_balance(market_name="sBTC")
    seth_balance = snx.spot.get_balance(market_name="sETH")

    print(
        f"""
    Perps accounts: {snx.perps.account_ids}
    Core accounts: {snx.core.account_ids}
        
    ETH: {eth_balance['eth']}
    WETH ({weth.address}): {eth_balance['weth']}
    sUSD: {susd_balance}
    USDC ({usdc.address}): {usdc_balance}
    BTC ({btc.address}): {btc_balance}
    sBTC: {sbtc_balance}
    sETH: {seth_balance}
    """
    )


status()


    Perps accounts: [170141183460469231731687303715884105727, 170141183460469231731687303715884105728, 170141183460469231731687303715884105729, 170141183460469231731687303715884105730, 170141183460469231731687303715884105731, 170141183460469231731687303715884105732, 170141183460469231731687303715884105733, 170141183460469231731687303715884105734, 170141183460469231731687303715884105735, 170141183460469231731687303715884105736, 170141183460469231731687303715884105737, 170141183460469231731687303715884105738, 170141183460469231731687303715884105739, 170141183460469231731687303715884105740, 170141183460469231731687303715884105741, 170141183460469231731687303715884105742, 170141183460469231731687303715884105743, 170141183460469231731687303715884105744, 170141183460469231731687303715884105745, 170141183460469231731687303715884105746, 170141183460469231731687303715884105747, 170141183460469231731687303715884105748, 170141183460469231731687303715884105749, 17014118346046923173168730371588410

In [14]:
snx.spot.markets_by_id

{0: {'market_id': 0,
  'market_name': 'sUSD',
  'contract': <web3._utils.datatypes.Contract at 0x13928b400>},
 1: {'market_id': 1,
  'market_name': 'sBTC',
  'symbol': 'BTC',
  'contract': <web3._utils.datatypes.Contract at 0x138dcc910>,
  'settlement_strategy': {'strategy_type': 1,
   'settlement_delay': 2,
   'settlement_window_duration': 60,
   'price_verification_contract': '0x0ea49b2e23387cd99cecebda52041c604e40992e',
   'feed_id': '0xe62df6c8b4a85fe1a67db44dc12de5db330f7ac66b72dc658afedf0f4a415b43',
   'url': 'https://api.synthetix.io/pyth-mainnet/api/get_vaa_ccip?data={data}',
   'settlement_reward': 0.5,
   'price_deviation_tolerance': 1.0,
   'minimum_usd_exchange_amount': 1e-06,
   'max_rounding_loss': 1e-06,
   'disabled': False}},
 2: {'market_id': 2,
  'market_name': 'sETH',
  'symbol': 'ETH',
  'contract': <web3._utils.datatypes.Contract at 0x13ada4d00>,
  'settlement_strategy': {'strategy_type': 1,
   'settlement_delay': 2,
   'settlement_window_duration': 60,
   'price_

In [ ]:
snx.perps.modify_collateral(0.1, market_id=2, submit=True, account_id=account_id)

In [ ]:
# commit_tx = snx.perps.commit_order(
#     0.1,
#     market_name="ETH",
#     settlement_strategy_id=0,
#     submit=True,
#     account_id=account_id,
# )
# commit_receipt = snx.wait(commit_tx)

# wait for the order settlement
settle_tx = snx.perps.settle_order(account_id=account_id, max_tx_tries=5, submit=True)
settle_receipt = snx.wait(settle_tx)

In [ ]:
snx.perps.get_margin_info(account_id=account_id)

In [ ]:
snx.perps.get_open_positions(account_id=account_id)

In [ ]:
perps.functions.debt(snx.perps.default_account_id).call() / 1e18

In [ ]:
perps.functions.getAccountCollateralIds(snx.perps.default_account_id).call()

In [ ]:
perps.functions.getCollateralAmount(snx.perps.default_account_id, 0).call() / 1e18

In [ ]:
perps.functions.getCollateralAmount(snx.perps.default_account_id, 1).call() / 1e18

In [ ]:
snx.perps.create_account(submit=True)

In [ ]:
new_account_id = snx.perps.account_ids[-1]

In [ ]:
collateral_id, collateral_name = snx.spot._resolve_market(
    market_id=None, market_name="sUSD"
)

# get starting collateral and sUSD balance
margin_info_start = snx.perps.get_margin_info(new_account_id)

# check allowance
allowance = snx.spot.get_allowance(
    snx.perps.market_proxy.address, market_name=collateral_name
)
if allowance < collateral_amount:
    approve_tx = snx.spot.approve(
        snx.perps.market_proxy.address, market_name=collateral_name, submit=True
    )
    snx.wait(approve_tx)

# modify collateral
modify_tx = snx.perps.modify_collateral(
    collateral_amount,
    market_name=collateral_name,
    account_id=new_account_id,
    submit=True,
)
snx.wait(modify_tx)

# check the result
margin_info_end = snx.perps.get_margin_info(new_account_id)

In [ ]:
snx.perps.get_margin_info(new_account_id)